## Example NoiseGrad and NoiseGrad++ text-classification

With a few lines of code we show how to use NoiseGrad and NoiseGrad++ with PyTorch, as a way to enhance text-classification explanation method.

• **Paper:** NoiseGrad: enhancing explanations by introducing stochasticity to model weights
• **Author:** Artem Sereda
• **Institution:** TU Berlin, ML Department, Understandable Machine Intelligence Lab
• **Date:** February, 2023


In [60]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from noisegrad import NoiseGrad, NoiseGradPlusPlus, utils, explainers
from functools import partial
from IPython.core.display import HTML

device = torch.device("cpu")

### Step 1. Load data and model

In [61]:
x_batch = [load_dataset("sst2")["test"]["sentence"][2]]
x_batch

Found cached dataset sst2 (/Users/artemsereda/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

['by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .']

In [62]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [63]:
input_ids = tokenizer(x_batch, return_tensors="pt")["input_ids"]

In [64]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy().tolist()[0])

In [65]:
y_batch = model(input_ids).logits.detach().argmax(axis=-1)

In [66]:
def embedding_lookup(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.int64, device=input_ids.device)
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    word_embeddings = model.get_input_embeddings()(input_ids)
    position_embeddings = model.get_position_embeddings()(position_ids)
    return word_embeddings + position_embeddings


x_batch_embeddings = embedding_lookup(input_ids)

### Step 2. Initialize methods

In [67]:
ng = NoiseGrad(n=5)
ng_pp = NoiseGradPlusPlus(n=5, m=5)

### Step 3. Get explanations

In [68]:
baseline_scores = explainers.explain_gradient_x_input(
    model, x_batch_embeddings, y_batch, attention_mask=None
).numpy()

In [69]:
ng_scores = ng.enhance_explanation(
    model,
    x_batch_embeddings,
    y_batch,
    explanation_fn=partial(explainers.explain_gradient_x_input, attention_mask=None),
).numpy()

NoiseGrad:   0%|          | 0/5 [00:00<?, ?it/s]

In [70]:
ng_pp_scores = ng_pp.enhance_explanation(
    model,
    x_batch_embeddings,
    y_batch,
    explanation_fn=partial(explainers.explain_gradient_x_input, attention_mask=None),
).numpy()

NoiseGrad++:   0%|          | 0/25 [00:00<?, ?it/s]

### Step 4. Visualize explanations

In [71]:
HTML(
    utils.visualise_explanations_as_html(
        [
            (tokens, baseline_scores[0]),
            (tokens, ng_scores[0]),
            (tokens, ng_pp_scores[0]),
        ],
        ignore_special_tokens=True,
        labels=["Baseline", "NoiseGrad", "NoiseGrad++"],
    )
)